In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf


In [3]:
# Load the dataset and split into training and validation sets
(train_ds, val_ds), info = tfds.load(
    "stanford_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True,
)

# Select a subset of the dataset
train_ds = train_ds.take(1000)
val_ds = val_ds.take(200)


# Preprocess the data
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    return image, label


train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Specify the batch size
batch_size = 32

# Shuffle and batch the data
train_ds = (
    train_ds.cache()
    .shuffle(1000)
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
val_ds = val_ds.cache().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Build the model
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(224, 224, 3)
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

# Train the model
epochs = 10
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

# Evaluate the model
test_ds, info = tfds.load(
    "stanford_dogs", split="test", with_info=True, as_supervised=True
)
test_ds = test_ds.take(200)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_loss, test_acc = model.evaluate(
    test_ds, steps=info.splits["test"].num_examples // batch_size
)
print("Test accuracy:", test_acc)


Epoch 1/10
32/32 [==============================] - 103s 3s/step - loss: -9332477.0000 - accuracy: 0.0100 - val_loss: -73307784.0000 - val_accuracy: 0.0050
Epoch 2/10
32/32 [==============================] - 51s 2s/step - loss: -1252032384.0000 - accuracy: 0.0100 - val_loss: -5156636672.0000 - val_accuracy: 0.0050
Epoch 3/10
32/32 [==============================] - 62s 2s/step - loss: -24603400192.0000 - accuracy: 0.0100 - val_loss: -71922573312.0000 - val_accuracy: 0.0050
Epoch 4/10
32/32 [==============================] - 60s 2s/step - loss: -212227768320.0000 - accuracy: 0.0100 - val_loss: -474883194880.0000 - val_accuracy: 0.0050
Epoch 5/10
32/32 [==============================] - 59s 2s/step - loss: -1036646088704.0000 - accuracy: 0.0100 - val_loss: -1986637856768.0000 - val_accuracy: 0.0050
Epoch 6/10
32/32 [==============================] - 60s 2s/step - loss: -3685091377152.0000 - accuracy: 0.0100 - val_loss: -6295967498240.0000 - val_accuracy: 0.0050
Epoch 7/10
32/32 [========

268/268 [==============================] - 8s 26ms/step - loss: -132179690520576.0000 - accuracy: 0.0250
Test accuracy: 0.02500000037252903
